# Lab 2: MapReduce with PySpark

## Objectives
In this lab, you will:
- Learn how PySpark processes data in a distributed system.
- Implement MapReduce operations like `map`, `reduceByKey`, and `sortBy`.
- Explore how partitioning affects performance.

### Key Exercises
1. Word Count
2. Grouping and Aggregation
3. Distributed Sorting
4. Partitioning and Performance Analysis

## Setup

### Prerequisites
- Python 3.x installed
- PySpark installed (use `pip install pyspark` if needed)

### Import Libraries
Run the cell below to import required libraries and initialize helper functions.

In [ ]:
from pyspark import SparkContext
import time
from functools import wraps

def timed(func):
    """Decorator to measure the execution time of a function."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function '{func.__name__}' executed in {end_time - start_time:.4f} seconds.")
        return result
    return wrapper

## Exercise 1: Word Count

### Objective
Count the occurrences of each word in a large text dataset using PySpark.

### Steps
1. Load the dataset `text_data.txt`.
2. Split each line into words.
3. Map each word to `(word, 1)`.
4. Use `reduceByKey` to aggregate word counts.
5. Collect and display the results.

In [ ]:
@timed
def word_count(file_path):
    sc = SparkContext("local", "WordCount")
    
    # Load the dataset
    rdd = sc.textFile(file_path)
    
    # Split lines into words
    words_rdd = rdd.flatMap(lambda line: line.split())
    
    # Map each word to (word, 1)
    pairs_rdd = words_rdd.map(lambda word: (word, 1))
    
    # Reduce by key to count occurrences
    word_counts_rdd = pairs_rdd.reduceByKey(lambda a, b: a + b)
    
    # Collect and return the results
    results = word_counts_rdd.collect()
    sc.stop()
    return results

# Run the Word Count exercise
results = word_count("text_data.txt")
print(results[:10])  # Print the first 10 word counts

## Exercise 2: Grouping and Aggregation

### Objective
Perform grouping and aggregation on a dataset of `(k, v)` pairs to compute the sum of values for each key.

### Steps
1. Load the dataset `group_data.txt`.
2. Parse each line into `(k, v)` pairs.
3. Use `reduceByKey` to aggregate values for each key.
4. Collect and display the results.

In [ ]:
@timed
def grouping_aggregation(file_path):
    sc = SparkContext("local", "GroupingAggregation")
    
    # Load the dataset
    rdd = sc.textFile(file_path)
    
    # Parse each line into (k, v) pairs
    pairs_rdd = rdd.map(lambda line: tuple(map(int, line.split(","))))
    
    # Use reduceByKey to sum values for each key
    aggregated_rdd = pairs_rdd.reduceByKey(lambda a, b: a + b)
    
    # Collect and return the results
    results = aggregated_rdd.collect()
    sc.stop()
    return results

# Run the Grouping and Aggregation exercise
grouped_results = grouping_aggregation("group_data.txt")
print(grouped_results)

## Exercise 3: Distributed Sorting

### Objective
Sort a dataset of integers distributed across the cluster.

### Steps
1. Load the dataset `sort_data.txt`.
2. Parse each line into integers.
3. Use `sortBy` to sort the dataset in ascending order.
4. Collect and display the sorted results.

In [ ]:
@timed
def distributed_sorting(file_path):
    sc = SparkContext("local", "DistributedSorting")
    
    # Load the dataset
    rdd = sc.textFile(file_path)
    
    # Parse lines into integers
    numbers_rdd = rdd.map(lambda line: int(line.strip()))
    
    # Sort the numbers
    sorted_rdd = numbers_rdd.sortBy(lambda x: x)
    
    # Collect and return the sorted results
    results = sorted_rdd.collect()
    sc.stop()
    return results

# Run the Distributed Sorting exercise
sorted_numbers = distributed_sorting("sort_data.txt")
print(sorted_numbers[:10])  # Print the first 10 sorted numbers

## Exercise 4: Partitioning and Performance Analysis

### Objective
Explore how partitioning affects the performance of distributed transformations.

### Steps
1. Load the dataset `text_data.txt`.
2. Repartition the dataset using `repartition`.
3. Apply a transformation (e.g., split lines into words).
4. Measure the execution time before and after repartitioning.

In [ ]:
@timed
def partitioning_analysis(file_path, num_partitions):
    sc = SparkContext("local", "PartitioningAnalysis")
    
    # Load the dataset
    rdd = sc.textFile(file_path)
    
    # Repartition the dataset
    repartitioned_rdd = rdd.repartition(num_partitions)
    
    # Apply a transformation (split lines into words)
    transformed_rdd = repartitioned_rdd.flatMap(lambda line: line.split())
    
    # Collect and return the results
    results = transformed_rdd.collect()
    sc.stop()
    return results

# Run the Partitioning Analysis exercise
partitioned_results = partitioning_analysis("text_data.txt", 10)
print(partitioned_results[:10])  # Print the first 10 results